# メディア展開データの基礎分析

データ可視化の前に、メディア展開データの [**基礎分析**](basic-analysis) を行いましょう。

- `mix_ae_crt.csv`：アニメ各話とマンガ原作者の関係をまとめたファイル
- `mix_ce_ac.csv`：マンガ各話とアニメ作品の関係をまとめたファイル

ただし、[マンガデータの基礎分析](../02/eda.ipynb)や[アニメデータの基礎分析](../04/an_eda.ipynb)と重複する分析は割愛します。
また、データの作成方法に関しては[メディア展開データの前処理](../appendix/mix_preprocess.ipynb)を参照ください。

## 初期設定

以降では、マンガ・アニメ・ゲームデータを可視化するための初期設定を行います。
なお、紙幅の都合のため、書籍版と一部構成が異なることにご注意ください。

### Import

必要なライブラリをImportします。

In [1]:
# warningsモジュールのインポート
import warnings

# データ解析や機械学習のライブラリ使用時の警告を非表示にする目的で警告を無視
# 本書の文脈では、可視化の学習に議論を集中させるために選択した
# ただし、学習以外の場面で、警告を無視する設定は推奨しない
warnings.filterwarnings("ignore")

In [2]:
# difflibモジュールのインポート
# 文字列間の類似度や差異を計算するためのユーティリティを提供
import difflib

# pathlibモジュールのインポート
# ファイルシステムのパスを扱う
from pathlib import Path

# typingモジュールからの型ヒント関連のインポート
# 関数やクラスの引数・返り値の型を注釈するためのツール
from typing import Optional

# numpy：数値計算ライブラリのインポート
# npという名前で参照可能
import numpy as np

# pandas：データ解析ライブラリのインポート
# pdという名前で参照可能
import pandas as pd

# tqdm_notebookのインポート
# Jupyter Notebook環境でのプログレスバー表示に使用
# tqdmという名前で参照可能
from tqdm import tqdm_notebook as tqdm

### 定数

本Notebookで用いる定数を定義します。
なお、Pythonにおける定数の扱いについては、[こちら](python-const)を参照ください。

In [3]:
# 読み込み対象ディレクトリの定義

# メディア展開データが保存されているディレクトリのパス
DIR_IN = Path("../../data/mix/input")

In [4]:
# 読み込み対象ファイル名の定義

# アニメ各話と原作マンガの作者者の対応関係に関するファイル
FN_AE_CRT = "mix_ae_crt.csv"

# マンガ各話とアニメ作品の対応関係に関するファイル
FN_CE_AC = "mix_ce_ac.csv"

### 関数

本Notebookで用いる関数を定義します。

In [5]:
def find_closest_match(row: pd.Series, df: pd.DataFrame) -> Optional[str]:
    """
    DataFrameの行に対して、指定されたアニメ作品名(`acname`)に最も近いマンガ作品名(`ccname`)を見つける
    ただし、`acname`がNaNの行は処理をスキップする

    Parameters
    ----------
    row : pd.Series
        処理するDataFrameの行。各行はアニメ作品名(`acname`)を含む
    df : pd.DataFrame
        検索対象のマンガ作品名(`ccname`)を含むDataFrame

    Returns
    -------
    Optional[str]
        最も類似度が高いマンガ作品名
        ただし、`acname`がNaNの場合はNoneを返す
    """
    # acnameがNaNの場合は処理をスキップ
    if pd.isna(row["acname"]):
        return None

    # acnameに最も近いccnameを返す
    closest_matches = difflib.get_close_matches(
        row["acname"], df["ccname"], n=1, cutoff=0.0
    )
    return closest_matches[0] if closest_matches else None

## `mix_ae_crt.csv`の基礎分析

`mix_ae_crt.csv`は、アニメ各話とその原作マンガの作者に関する情報を一つに集約したファイルです。
次のような列を持ちます。

- `aeid`：Anime Episode ID。アニメ各話ID
- `aename`：Anime Episode NAME。アニメ各話名。
- `date`：アニメ各話の放送日
- `aeno`：アニメ各話数
- `acid`：Anime Collection ID。アニメ作品ID
- `acname`：Anime Collection NAME。アニメ作品名
- `asid`：Anime Series ID。アニメシリーズID
- `n_ae`：当該アニメ作品の合計各話数
- `first_date`：当該アニメ作品の最初の放送日
- `last_date`：当該アニメ作品の最後の放送日
- `ccid`：Comic Collection ID。マンガ作品ID
- `ccname`：Comic Collection NAME。マンガ作品名
- `mcname`：Magazine Collection NAME。マンガ雑誌名
- `crtid`：CReaTor ID。マンガ作者ID
- `crtname`：CreaTor NAME。マンガ作者名

まず、分析対象のファイルを読み込みましょう。

In [6]:
# pandasのread_csv関数でCSVファイルの読み込み
df_ae_crt = pd.read_csv(DIR_IN / FN_AE_CRT)

### 全体像の把握

`df_ae_crt`を概観してみましょう。

In [7]:
# df_ae_crtデータフレームの先頭5行をTで転置して表示
# 列が一定以上あるとき、転置すると見やすくなることがある
df_ae_crt.head().T

,0,1,2,3,4
aeid,M53249,M53250,M53251,M53252,M53253
aename,共 振 バイブレーション,発 動 トランス,悪 夢 ナイトメア,迎 撃 インターセプト,反 撃 カウンターアタック
date,2001-04-08,2001-04-15,2001-04-22,2001-04-29,2001-05-06
aeno,1,2,3,4,5
acid,C10003,C10003,C10003,C10003,C10003
acname,PROJECT ARMS,PROJECT ARMS,PROJECT ARMS,PROJECT ARMS,PROJECT ARMS
asid,C2484,C2484,C2484,C2484,C2484
n_ae,26,26,26,26,26
first_date,2001-04-08,2001-04-08,2001-04-08,2001-04-08,2001-04-08
last_date,2001-09-30,2001-09-30,2001-09-30,2001-09-30,2001-09-30


形状を確認してみましょう。

In [8]:
# df_ae_crtデータフレームの形状（行数・列数）を取得
# shape属性を使用すると、(行数, 列数)の形で結果が返される
df_ae_crt.shape

(12700, 15)

欠損値の合計数と、全体に占める割合を確認してみましょう。

In [9]:
# df_ae_crtデータフレーム内の欠損値（NaN）の情報を集計
# isna()メソッドを使用して欠損値の場所をTrueとして特定
# その後、agg()メソッドを使用して、各列における欠損値の合計と平均を計算しTで転置
df_ae_crt.isna().agg([sum, "mean"]).T

,sum,mean
aeid,0.0,0.000000
aename,26.0,0.002047
date,0.0,0.000000
aeno,104.0,0.008189
acid,0.0,0.000000
acname,0.0,0.000000
asid,0.0,0.000000
n_ae,0.0,0.000000
first_date,0.0,0.000000
last_date,0.0,0.000000


`aename`に約0.2%ほど、そして`aeno`に約0.8%ほど欠損があることがわかります。

記述統計情報を確認してみましょう。

In [10]:
# df_ae_crtデータフレームの記述統計情報を取得
# describe()メソッドを使用して、各列の基本的な統計量を表示
df_ae_crt.describe()

,n_ae
count,12700.000000
mean,184.338110
std,236.538762
min,1.000000
25%,30.000000
50%,75.000000
75%,198.000000
max,783.000000


ユニークな値の数を確認してみましょう。

In [11]:
# df_ae_crtデータフレームの各列に対してユニークな値の数をカウント
# nunique()メソッドを使用して、各列のユニークな値の数を計算
# その後、結果を新しいデータフレームとして整形し、列名を`nunique`とする
df_ae_crt.nunique().reset_index(name="nunique")

,index,nunique
0,aeid,11407
1,aename,11315
2,date,5926
3,aeno,1076
4,acid,238
5,acname,238
6,asid,152
7,n_ae,75
8,first_date,214
9,last_date,216


行数より `aeid` や `crtid` のユニーク数が少ないので、それぞれ単独では重複が存在することがわかります。

In [12]:
# `aeid`と`crtid`の組み合わせで重複する行がないことを確認
assert df_ae_crt.duplicated(subset=["aeid", "crtid"]).sum() == 0

`aeid`と`crtid`の組合せに関して重複がないことを確認できました。

以降、カラムごとに少し踏み込んだ分析を行います。

### `date`列の深掘り

`date`（放送日）について分析してみましょう。

In [13]:
# df_aeデータフレームの`date`列から最も古い日付と最も新しい日付を取得
# min()メソッドを使用して、最も古い日付を抽出
# max()メソッドを使用して、最も新しい日付を抽出
df_ae_crt["date"].min(), df_ae_crt["date"].max()

('1990-01-14', '2017-09-30')

このデータは、1990年1月14日から2017年9月30日に放送された、四大少年誌のマンガ作品を原作とするアニメ各話を対象としていることがわかります。

In [14]:
# `date`列を日付型に変換
# pd.to_datetime()メソッドを使用して、文字列型の日付を日付型(datetime)に変換
df_ae_crt["date"] = pd.to_datetime(df_ae_crt["date"])

# 年に関する情報を新しい`year`列として追加
# dt.yearを使用して、日付から年のみを取得
df_ae_crt["year"] = df_ae_crt["date"].dt.year

# 月に関する情報を新しい`month`列として追加
# dt.monthを使用して、日付から月のみを取得
df_ae_crt["month"] = df_ae_crt["date"].dt.month

# 曜日に関する情報を新しい`weekday`列として追加
# dt.weekdayを使用して、日付から曜日の情報を数値(0:月曜日, 6:日曜日)として取得
df_ae_crt["weekday"] = df_ae_crt["date"].dt.weekday

In [15]:
# `year`（年）ごとに各カラムのユニークな値の数を集計
# 具体的には、groupbyメソッドで`year`を基準にグループ化し、
# nuniqueメソッドを使用して各カラムのユニークな値の数を計算
# その後、reset_indexメソッドでインデックスをリセットし、データフレームとして結果を返す
df_ae_crt.groupby("year")[["month", "asid", "acid", "aeid"]].nunique().reset_index()

,year,month,asid,acid,aeid
0,1990,5,3,3,41
1,1991,12,6,6,158
2,1992,12,4,4,102
3,1993,12,6,6,153
4,1994,12,9,9,257
5,1995,12,5,5,143
6,1996,12,6,6,136
7,1997,12,8,8,186
8,1998,12,11,11,316
9,1999,12,12,12,223


`1990`年から`2017`年まで毎年、四大少年誌を原作とする何らかのアニメ各話が放送されていたことがわかります。

In [16]:
# `weekday`（曜日）ごとに各カラムのユニークな値の数を集計
# 具体的には、groupbyメソッドで`weekday`を基準にグループ化し、
# nuniqueメソッドを使用して各カラムのユニークな値の数を計算
# その後、reset_indexメソッドでインデックスをリセットし、データフレームとして結果を返す
df_ae_crt.groupby("weekday")[["asid", "acid", "aeid"]].nunique().reset_index()

,weekday,asid,acid,aeid
0,0,26,32,1289
1,1,37,45,1323
2,2,35,39,1581
3,3,26,30,1213
4,4,29,33,446
5,5,49,68,2510
6,6,53,72,3045


全ての曜日で四大少年誌を原作としたアニメ作品の放送実績があることがわかりました。
特に土曜（`5`）と日曜（`6`）でアニメシリーズ数（`asid`）、アニメ作品数（`acid`）、そして各話数（`aeid`）が多くなっていますが、これは[アニメデータの基礎分析](../04/an_eda.ipynb)と同様の傾向です。

### `aeid`、`aename`、`aeno`列の深掘り

`aeid`（アニメ各話ID）、`aename`（アニメ各話）、`aeno`（アニメ各話数）について分析してみましょう。

まず、マンガ雑誌ごとのアニメ各話数のシェアを集計してみます。

In [17]:
# 原作マンガの属するマンガ雑誌ごとに、アニメ各話数を集計
df_ae_crt.groupby("mcname")["aeid"].nunique().reset_index(name="n_ae")

,mcname,n_ae
0,週刊少年サンデー,2655
1,週刊少年ジャンプ,6393
2,週刊少年チャンピオン,433
3,週刊少年マガジン,1926


アニメ各話数が最も多いのは`週刊少年ジャンプ`であることがわかりました。
ただし、アニメ各話自体の区切り方がデータ内で統一されていないことに注意が必要です。
つまり、一度の **放送枠** ( **Anime Slot** ) に対して複数エピソードが放送されるとき、それぞれを別の「アニメ各話」とするか、一つにまとめるかはアニメ作品に依存しています。
そこで、アニメ作品と日付の組合せを放送枠と定義し、このユニーク数を集計することを考えます。

In [18]:
# atsname（Anime Time Slot NAME)としてacidとdateを結合した文字列を追加
df_ae_crt["atsname"] = df_ae_crt["acid"] + "_" + df_ae_crt["date"].astype(str)
# mcnameごとにユニークなアニメ放送枠数を集計
df_ae_crt.groupby("mcname")["atsname"].nunique().reset_index(name="n_ats")

,mcname,n_ats
0,週刊少年サンデー,2580
1,週刊少年ジャンプ,6188
2,週刊少年チャンピオン,417
3,週刊少年マガジン,1838


放送枠数で集計した場合でも、傾向に大きな変化はありませんでした。

次に、四大少年誌を原作としたアニメ各話の文字数に注目してみます。

In [19]:
# aename列の文字列の長さを算出し、l_aename列として格納
df_ae_crt["l_aename"] = df_ae_crt["aename"].str.len()

In [20]:
# l_aename列の統計量を算出
df_ae_crt["l_aename"].describe().reset_index()

,index,l_aename
0,count,12674.000000
1,mean,14.680054
2,std,9.101988
3,min,1.000000
4,25%,9.000000
5,50%,13.000000
6,75%,18.000000
7,max,200.000000


`aename`は平均約15文字、最大で`200`文字のものがあることがわかりました。
では、どのようなアニメ各話の文字数が多いのか確認してみましょう。

In [21]:
# l_aenameで降順ソートし、特定の列のみ抽出して冒頭5レコードを表示
df_ae_crt.sort_values("l_aename", ascending=False)[
    ["acname", "aename", "l_aename"]
].head()

,acname,aename,l_aename
2170,School Rumble,[突然の「さよなら」…迷い 込んだラビリンス…あなたはだれ? …教えて。「すれちがい」「片想...,200.0
4762,銀魂,仕事のグチは家でこぼさず外でこぼせ!って言うからちょっとこぼさせてもらうけどね「侍の国」僕ら...,183.0
3926,【俗・】さよなら絶望先生,絶望ファイト絶望の大逆転戦法!（3/2）一人より女夫の方がええいうことでっしゃろ（3/2）絶...,130.0
6434,バクマン。[第3期],デラマン。[第1回]（9/23） メモリーズ #1 “最高と秋人”（9/23） メモリーズ ...,129.0
6433,バクマン。[第3期],デラマン。[第1回]（9/23） メモリーズ #1 “最高と秋人”（9/23） メモリーズ ...,129.0


四大少年誌に掲載されたマンガ作品を原作としたアニメ作品のうち、各話名が最も長かったのは、`School Rumble`の`[突然の「さよなら」…迷い 込んだラビリンス…あなたはだれ? …教えて。「すれちがい」「片想い」 とどけ、ボクの気持ち。とどけ、ワ タシの想い。たぶん一度しかない季 節、青春の1ページ。これが最後の チャンス、確かめたい…キミの気持 ち。伝わる言葉、伝わらない想い。 あの日の告白、永遠の一日、だけど …いつまでも続いていく、わたした ちの「いま」。そして明日へ… 「スクールランブルフォーエバー」]`であることがわかりました。次に長いのは`銀魂`の`仕事のグチは家でこぼさず外でこぼせ!って言うからちょっとこぼさせてもらうけどね「侍の国」僕らの国がそう呼ばれていたのは今は昔の話…とか言って始まったこのアニメもはや一年半△あんな事こんな事いろんな事があったよね△で、そろそろ色々振り返ってもいいかなーと思ったのに「チェッ、なんだよ総集編かよ、手抜きじゃね?」とかアニメだって作るの大変なんだから文句言うのやめなさい!`です。

では、平均的な各話名が長いアニメ作品は何でしょうか？

In [22]:
# df_ae_crtデータフレームを"acid"と"acname"のカラムでグループ化し、
# 各グループの"l_aename"カラムに対する基本統計を計算してDataFrameに変換
df_tmp = df_ae_crt.groupby(["acid", "acname"])["l_aename"].describe().reset_index()

# 得られた統計要約df_tmpを"mean"カラム（平均値）に基づいて降順にソートし上位5行を表示
df_tmp.sort_values("mean", ascending=False).head()

,acid,acname,count,mean,std,min,25%,50%,75%,max
106,C13492,もっと ToLOVEる,24.0,71.000000,3.867366,66.0,67.75,71.0,74.0,79.0
73,C12181,生徒会 役員共,13.0,62.076923,17.303475,14.0,58.00,70.0,72.0,76.0
143,C14951,生徒会 役員共＊,14.0,48.928571,10.608156,29.0,43.75,48.0,54.0,69.0
57,C11615,【俗・】さよなら絶望先生,13.0,45.230769,26.739963,24.0,34.00,39.0,41.0,130.0
118,C13840,侵略!? イカ娘 The Invader comes from the bottom of ...,12.0,43.083333,2.998737,37.0,41.75,43.5,45.0,48.0


平均的なアニメ各話名の長さを集計すると、最も長いのは`もっと ToLOVEる`であることがわかりました。
意外な結果となったので、各話名をいくつか表示してみましょう。

In [23]:
# df_ae_crtから"acname"が"もっと ToLOVEる"に一致する行をフィルタリング
# その結果から"aename"カラムを選択し、最初の要素を取得
df_ae_crt[df_ae_crt["acname"] == "もっと ToLOVEる"]["aename"].iloc[0]

'もっとトラブル01\u3000もう一度ここから（10/6）\u3000もっとトラブル02\u3000お風呂場戦争（10/6）\u3000もっとトラブル03\u3000チクタク チクタク 恋の音▽*'

アニメ作品`もっと ToLOVEる`では複数話が一つの「各話」としてまとめられており、それぞれのサブタイトルを結合した「各話名」が格納されていることがわかりました。

### `acid`、`acname`列の深掘り

`acid`（アニメ作品ID）と`acname`（アニメ作品名）について分析してみましょう。

マンガ雑誌ごとのアニメ作品数を集計してみます。

In [24]:
# mcnameごとのユニークなacid数を集計し、n_ac列に結果を格納して表示
df_ae_crt.groupby("mcname")["acid"].nunique().reset_index(name="n_ac")

,mcname,n_ac
0,週刊少年サンデー,56
1,週刊少年ジャンプ,104
2,週刊少年チャンピオン,20
3,週刊少年マガジン,58


マンガ原作のアニメ作品数に関しても、`週刊少年ジャンプ`が最も多いことがわかりました。
では、アニメ放送枠数の統計情報はどうでしょうか？

In [25]:
# 'mcname'（マンガ雑誌名）と'acid'（アニメ作品ID）でグループ化し、
# 各グループ内でユニークな'atsname'（放送枠名）の数を集計
df_tmp = (
    df_ae_crt.groupby(["mcname", "acid"])["atsname"].nunique().reset_index(name="n_ats")
)

# 上記で集計したデータを'mcname'（マンガ雑誌名）でさらにグループ化し、
# 各マンガ雑誌名ごとに放送枠の数（'n_ats'）の要約統計量を計算
df_tmp.groupby("mcname")["n_ats"].describe().reset_index()

,mcname,count,mean,std,min,25%,50%,75%,max
0,週刊少年サンデー,56.0,46.071429,97.280823,1.0,13.0,26.0,46.25,727.0
1,週刊少年ジャンプ,104.0,59.500000,98.698718,1.0,13.0,26.0,61.00,759.0
2,週刊少年チャンピオン,20.0,20.850000,12.832012,1.0,12.0,20.5,25.25,60.0
3,週刊少年マガジン,58.0,31.689655,31.557915,1.0,13.0,25.0,38.00,175.0


上の表は、アニメ作品ごとの放送枠数の統計情報を、その原作マンガが掲載されたマンガ雑誌別に集計したものです。

平均値（`mean`）という観点では`週刊少年ジャンプ`が最も多く、約60枠となっています。
しかし中央値(`50%`)を見ると **26枠** つまり2クール分となっており、他誌を原作としたアニメ作品と大きな差はありません。
長寿アニメ作品の存在により、平均値が引き上げられていることが推察されます。

### `asid`列の深堀り

`asid`（アニメシリーズID）について分析してみましょう。

まず、マンガ雑誌ごとのアニメシリーズ数を集計してみます。

In [26]:
# mcnameごとのユニークなasid数と（参考情報として）acid数を集計して表示
df_ae_crt.groupby("mcname").agg(
    n_as=("asid", "nunique"), n_ac=("acid", "nunique")
).reset_index()

,mcname,n_as,n_ac
0,週刊少年サンデー,35,56
1,週刊少年ジャンプ,63,104
2,週刊少年チャンピオン,12,20
3,週刊少年マガジン,42,58


マンガ原作のアニメシリーズ数という観点でも、`週刊少年ジャンプ`が最も多いことがわかりました。
では、アニメ放送枠の統計情報はどうでしょうか。

In [27]:
# 'mcname'（マンガ雑誌名）と'asid'（アニメシリーズID）でグループ化し、
# 各グループ内でユニークな'atsname'（放送枠名）の数を集計
df_tmp = (
    df_ae_crt.groupby(["mcname", "asid"])["atsname"].nunique().reset_index(name="n_ats")
)

# 上記で集計したデータを'mcname'（マンガ雑誌名）でさらにグループ化し、
# 各マンガ雑誌名ごとに放送枠の数（'n_ats'）の要約統計量を計算
df_tmp.groupby("mcname")["n_ats"].describe().reset_index()

,mcname,count,mean,std,min,25%,50%,75%,max
0,週刊少年サンデー,35.0,73.714286,125.158857,12.0,26.50,50.0,56.5,752.0
1,週刊少年ジャンプ,63.0,98.222222,134.832446,9.0,31.00,53.0,98.5,759.0
2,週刊少年チャンピオン,12.0,34.750000,26.385688,12.0,13.00,25.0,49.5,87.0
3,週刊少年マガジン,42.0,43.761905,51.435875,11.0,13.75,26.0,50.5,277.0


上の表は、アニメシリーズごとの放送枠数の統計情報を、その原作マンガが掲載されたマンガ雑誌別に集計したものです。

アニメ作品単位で集計した場合と異なり、平均値（`mean`）だけでなく中央値（`50%`）においても雑誌（`mcname`）間で違いが生まれました。
具体的には、`週刊少年サンデー`と`週刊少年ジャンプ`が、他誌と比較して長期アニメシリーズを抱えているように見えます[^statistic]。

[^statistic]: 原作マンガが掲載されたマンガ雑誌によってアニメシリーズの合計放送枠数が異なるように見えますが、現時点では **偶然そうなった** 可能性を否定できません。本書のスコープ外ですのでこれ以上深堀りはしませんが、このような問題に対処するアプローチの一つとして **統計的仮説検定** {cite}`ut1991`があります。興味のある方は調べてみると良いでしょう。

### `n_ae`列の深掘り

`n_ae`（アニメ各話数）について分析してみましょう。

まず、四大少年誌を原作としたアニメ作品の各話数に関して、要約統計量を計算します。
ただし、`df_ae_crt`自体は`aeid`を一行とした`DataFrame`ですので、事前に`acid`の重複を除外しておく必要があります。

In [28]:
# 'date'列でデータを昇順にソートし、新しいインデックスを付与
# これにより、日付が古い順にデータが並び替えられる
df_ae_crt = df_ae_crt.sort_values("date", ignore_index=True)

# 'acid'列で重複するレコードを削除し、新しいインデックスを付与
# 各アニメ作品名('acid')に対して、最初に出現するレコードだけが保持される
df_ac_crt = df_ae_crt.drop_duplicates(subset=["acid"], ignore_index=True)

In [29]:
# n_ae列の要約統計量を算出
# reset_index()関数で、結果のインデックスをリセットして、それを新しいデータフレームとして取得
df_ac_crt["n_ae"].describe().reset_index()

,index,n_ae
0,count,238.000000
1,mean,47.928571
2,std,85.058723
3,min,1.000000
4,25%,13.000000
5,50%,26.000000
6,75%,50.000000
7,max,783.000000


四大少年誌を原作とするアニメ作品の話数は、平均値が約48話、中央値が約26話であることがわかりました。
では、マンガ雑誌別に集計するとどうなるでしょうか？

In [30]:
# n_ae列の要約統計量を算出
df_ac_crt.groupby("mcname")["n_ae"].describe()

,count,mean,std,min,25%,50%,75%,max
mcname,,,,,,,,
週刊少年サンデー,56.0,47.410714,97.549573,1.0,20.50,26.0,50.00,729.0
週刊少年ジャンプ,104.0,61.471154,102.619096,1.0,13.00,26.0,61.25,783.0
週刊少年チャンピオン,20.0,21.650000,13.152046,1.0,12.75,23.5,25.25,63.0
週刊少年マガジン,58.0,33.206897,31.871319,2.0,13.00,26.0,44.25,175.0


マンガ雑誌ごとに違いがあるように見えます。

### `first_date`、`last_date`列の深掘り

`first_date`（最初の放送日）および`last_date`（最後の放送日）について分析してみましょう。

まず、アニメ作品ごとに`first_date`と`last_date`を格納した`DataFrame`を作成します。

In [31]:
# df_ae_crtデータフレームから重複する行を削除し、指定された列だけを選択して新しいデータフレームdf_ac_datesを作成
# 'acid'列を基準に重複を排除し、ignore_index=Trueで新しいインデックスを割り当て
df_ac_dates = df_ae_crt.drop_duplicates(subset=["acid"], ignore_index=True)[
    ["mcname", "acname", "first_date", "last_date", "n_ae"]
]

# 'first_date'列と'last_date'列のデータを日付型に変換
df_ac_dates["first_date"] = pd.to_datetime(df_ac_dates["first_date"])
df_ac_dates["last_date"] = pd.to_datetime(df_ac_dates["last_date"])

# 'last_date'と'first_date'の差（期間）を計算して新しい列'duration'に保存
df_ac_dates["duration"] = df_ac_dates["last_date"] - df_ac_dates["first_date"]

`first_date`と`last_date`に関して要約統計量を表示してみましょう。

In [32]:
# first_dateとlast_dateの要約統計量を表示
df_ac_dates[["first_date", "last_date"]].describe()

,first_date,last_date
count,238,238
mean,2007-04-28 10:11:05.546218496,2008-04-15 19:09:34.789915904
min,1990-01-14 00:00:00,1990-12-24 00:00:00
25%,2002-05-18 00:00:00,2004-03-21 12:00:00
50%,2007-10-01 12:00:00,2008-08-29 00:00:00
75%,2013-06-16 06:00:00,2014-03-29 18:00:00
max,2017-07-05 00:00:00,2017-09-30 00:00:00


`first_date`に関する平均値は`2007-04-28`付近、`last_date`に関する平均値は`2008-04-15`付近にあることがわかりました。

次は、`duration`（放送期間）に関する要約統計量を確認してみましょう

In [33]:
# duration列の要約統計量を取得し、reset_indexでDataFrame化して表示
df_ac_dates["duration"].describe().reset_index()

,index,duration
0,count,238
1,mean,353 days 08:58:29.243697480
2,std,683 days 01:53:27.886971808
3,min,0 days 00:00:00
4,25%,84 days 00:00:00
5,50%,175 days 00:00:00
6,75%,356 days 18:00:00
7,max,6276 days 00:00:00


`duration`に関する平均値は`353`日付近にあることがわかりました。
ただし、複数クールに渡るアニメ作品が一つの`acid`で管理されている場合、アニメ各話数（`n_ae`）に対して放送期間（`duration`）が長くなることに注意しましょう。

試しに、`duration`を`n_ae`で割り放送間隔（`interval`）を算出してみましょう。

In [34]:
# 'duration'列を'n_ae'列で割って、各レコードについての平均間隔（'interval'）を算出
df_ac_dates["interval"] = df_ac_dates["duration"] / df_ac_dates["n_ae"]

# 'interval'列に対して要約統計量を取得し、reset_index()関数でDataFrame化して表示
df_ac_dates["interval"].describe().reset_index()

,index,interval
0,count,238
1,mean,7 days 01:29:51.880598083
2,std,4 days 00:30:03.634059959
3,min,0 days 00:00:00
4,25%,6 days 11:04:36.923076923
5,50%,6 days 19:41:32.307692307
6,75%,7 days 05:34:17.142857142
7,max,54 days 16:00:00


四大少年誌を原作としたほとんどのアニメ作品は、想定通り放送間隔が7日、つまり週1回のレギュラー放送であることがわかりました。
放送間隔が極端な値を取ったアニメ作品を確認してみましょう。

In [35]:
# intervalで昇順にソートし、上位5作品を表示
df_ac_dates.sort_values("interval").head()

,mcname,acname,first_date,last_date,n_ae,duration,interval
153,週刊少年チャンピオン,みつ どもえ 特別編,2011-03-07,2011-03-07,1,0 days,0 days
94,週刊少年ジャンプ,こちら葛飾区 亀有公園前 派出所 [スペシャル] 走れ! 両津式チンチン電車 ～思い出の大次郎号～,2006-04-02,2006-04-02,1,0 days,0 days
142,週刊少年サンデー,まじっく快斗 KID THE PHANTOM THIEF 第1話 「蘇る怪盗」,2010-04-17,2010-04-17,1,0 days,0 days
103,週刊少年ジャンプ,こちら葛飾区 亀有公園前 派出所 [スペシャル] 両津の 浅草リニューアル 大作戦!! ～あ...,2006-09-24,2006-09-24,1,0 days,0 days
215,週刊少年マガジン,金田一 少年の 事件簿R Special 明智警部の事件簿,2015-12-26,2015-12-26,2,0 days,0 days


`interval`が0の作品として、単話作品や特別企画作品が存在することがわかりました。では、`interval`が大きい作品はどうでしょうか？

In [36]:
# intervalで降順にソートし、上位5作品を表示
df_ac_dates.sort_values("interval", ascending=False).head()

,mcname,acname,first_date,last_date,n_ae,duration,interval
93,週刊少年ジャンプ,聖闘士星矢冥王 ハーデス冥界編[後章],2006-01-21,2006-12-15,6,328 days,54 days 16:00:00
0,週刊少年ジャンプ,シティーハンター3,1990-01-14,1990-12-24,13,344 days,26 days 11:04:36.923076923
124,週刊少年ジャンプ,聖闘士星矢冥王 ハーデスエリシオン編,2008-03-07,2008-08-01,6,147 days,24 days 12:00:00
169,週刊少年ジャンプ,銀魂'[延長戦],2012-10-04,2013-03-28,13,175 days,13 days 11:04:36.923076923
90,週刊少年サンデー,メジャー[第2期],2005-12-10,2007-01-03,30,389 days,12 days 23:12:00


四大少年誌を原作としたアニメ作品のうち、最も`interval`が長いのは`聖闘士星矢冥王 ハーデス冥界編[後章]`でした。
こちらはOriginal Video Animation（OVA）を軸とした作品とされているため、やや特殊と言えるかもしれません。
気になるのはレギュラーシリーズだったはずの`シティーハンター3`です。

In [37]:
# acnameがシティーハンター3と一致するレコードを抽出し、一部の列を表示
df_ae_crt[df_ae_crt["acname"] == "シティーハンター3"][["aeno", "aename", "date"]]

,aeno,aename,date
0,12,（126） グッバイCITY さよならの 贈りもの （前 編）,1990-01-14
1,13,（127） グッバイCITY さよならの 贈りもの （後 編）,1990-01-21
9,1,（115） 脱モッコリ宣言！ XYZは世界を救う,1990-10-15
12,2,（116） 天下の恋愛現行犯！ 美人弁護士をくどく法,1990-10-22
15,3,"（117） 香もプッツン！ 僚*と令嬢代打結婚物語""""",1990-10-29
17,4,（118） 危ない探偵ごっこ! お嬢さんにパイソンを （前 編）,1990-11-05
20,5,（119） 危ない探偵ごっこ! お嬢さんにパイソンを （後 編）,1990-11-12
23,6,（120） がんこな海坊主! ジェラシー子猫物語,1990-11-19
26,7,（121） 恋はダイビング! 美女が水着に着がえたら,1990-11-26
29,8,（122） 僚*って何者？ 女子大生もスリルにメロメロ,1990-12-03


`aeno`を見る限り、`aename`は`f"（{累積話数}）{各話タイトル}"`という構成であると考えるのが自然でしょう。
この仮定が正しいとすると、当該シーズンの12-13話と1-11話の順序が入れ替わっているように見えます。
[Wikipedia](https://ja.wikipedia.org/wiki/%E3%82%B7%E3%83%86%E3%82%A3%E3%83%BC%E3%83%8F%E3%83%B3%E3%82%BF%E3%83%BC_(%E3%82%A2%E3%83%8B%E3%83%A1)#%E3%82%B7%E3%83%86%E3%82%A3%E3%83%BC%E3%83%8F%E3%83%B3%E3%82%BF%E3%83%BC3)等の情報を考慮すると、元データでは **1話目から11話目までの放送年を1年誤っている** 可能性が考えられます。

### `ccid`、`ccname`列の深掘り

`ccid`（マンガ作品ID）および`ccname`（マンガ作品名）について分析してみましょう。
ここでは特に、`ccname`と`acname`の一致率に注目します。

In [38]:
# df_ae_crtから重複する'acid'（アニメ作品名）を削除し、
# 'ccname'と'acname'の列だけを選択して新しいデータフレームを作成
df_ccname_acname = df_ae_crt.drop_duplicates(subset=["acid"])[
    ["mcname", "ccname", "acname"]
].reset_index(drop=True)

# 新しいデータフレームに対して、'ccname'と'acname'の文字列間の類似度を計算し、
# その結果を'similarity'列として追加
# ここで、1は完全一致、0は全く異なる場合を意味する
df_ccname_acname["similarity"] = df_ccname_acname.apply(
    lambda row: difflib.SequenceMatcher(None, row["ccname"], row["acname"]).ratio(),
    axis=1,
)

In [39]:
# 類似度が低いアニメ作品を10レコード表示
df_ccname_acname.sort_values(["similarity", "ccname"]).head(10)

,mcname,ccname,acname,similarity
194,週刊少年ジャンプ,DRAGON BALL,ドラゴンボール改[第2期],0.000000
80,週刊少年サンデー,MAJOR,メジャー,0.000000
90,週刊少年サンデー,MAJOR,メジャー[第2期],0.000000
110,週刊少年サンデー,MAJOR,メジャー[第3期],0.000000
121,週刊少年サンデー,MAJOR,メジャー[第4期],0.000000
130,週刊少年サンデー,MAJOR,メジャー[第5期],0.000000
141,週刊少年サンデー,MAJOR,メジャー[第6期],0.000000
75,週刊少年マガジン,スクールランブル,School Rumble,0.000000
95,週刊少年マガジン,スクールランブル,School Rumble 二学期,0.000000
34,週刊少年ジャンプ,HUNTER×HUNTER,ハンター×ハンター,0.090909


原作マンガ作品名（`ccname`）とアニメ作品名（`acname`）が1文字も共通していないものが多く存在します。
`similarity`の要約統計量を計算してみましょう。

In [40]:
# df_acname_ccnameデータフレームの'similarity'列を選択し、要約統計量を取得
# reset_index()関数で、結果のインデックスをリセットして、それを新しいデータフレームとして取得
df_ccname_acname["similarity"].describe().reset_index()

,index,similarity
0,count,238.000000
1,mean,0.723755
2,std,0.287821
3,min,0.000000
4,25%,0.533333
5,50%,0.800000
6,75%,1.000000
7,max,1.000000


類似度の平均値は約0.73であり、中央値は約0.80であることがわかりました。
では、マンガ雑誌別の傾向を見てみましょう。

In [41]:
# mcnameごとにsimilarity列の要約統計量を算出
df_ccname_acname.groupby("mcname")["similarity"].describe()

,count,mean,std,min,25%,50%,75%,max
mcname,,,,,,,,
週刊少年サンデー,56.0,0.659172,0.349151,0.0,0.369048,0.732143,1.0,1.0
週刊少年ジャンプ,104.0,0.768607,0.234002,0.0,0.608037,0.800000,1.0,1.0
週刊少年チャンピオン,20.0,0.750598,0.273587,0.2,0.571429,0.850000,1.0,1.0
週刊少年マガジン,58.0,0.696430,0.306820,0.0,0.463801,0.800000,1.0,1.0


`週刊少年サンデー`を原作としたアニメ作品に関して、比較的一致度が低いことがわかりました。具体的例を見てみましょう。

In [42]:
# mcnameが週刊少年サンデーに一致するレコードに対して、similarityを基準に昇順ソート
# 上位10作品を表示
df_ccname_acname[df_ccname_acname["mcname"] == "週刊少年サンデー"].sort_values(
    "similarity"
).head(10)

,mcname,ccname,acname,similarity
121,週刊少年サンデー,MAJOR,メジャー[第4期],0.000000
80,週刊少年サンデー,MAJOR,メジャー,0.000000
110,週刊少年サンデー,MAJOR,メジャー[第3期],0.000000
130,週刊少年サンデー,MAJOR,メジャー[第5期],0.000000
141,週刊少年サンデー,MAJOR,メジャー[第6期],0.000000
90,週刊少年サンデー,MAJOR,メジャー[第2期],0.000000
9,週刊少年サンデー,ゴーストスイーパー美神 極楽大作戦!!,GS美神,0.173913
170,週刊少年サンデー,ハヤテのごとく!,ハヤテのごとく! Hayate the combat buttler CAN'T TAKE ...,0.231884
206,週刊少年サンデー,電波教師,電波教師 HE IS AN ULTIMATE TEACHER,0.235294
54,週刊少年サンデー,ARMS,PROJECT ARMS The 2nd Chapter,0.250000


`MAJOR`についてはもう触れましたが、他にも`ゴーストスイーパー美神 極楽大作戦!!`が`GS美神`、`ハヤテのごとく!`が`ハヤテのごとく! Hayate the combat buttler CAN'T TAKE MY EYES OFF YOU`になっています。
マンガ作品名（`ccname`）とアニメ作品名（`acname`）だけでは、機械的な紐づけは非常に困難です。

### `mcname`列の深堀り

`mcname`（マンガ雑誌名）について分析してみましょう。
`mcname`でグループ化し、各列のユニーク数を集計してみます。

In [43]:
# df_ae_crtデータフレームを'mcname'列に基づいてグループ化し、
# [[...]].nunique()で、指定された列について各グループ内のユニークな値の数を計算
df_ae_crt.groupby("mcname")[["aeid", "acid", "asid", "ccid", "crtid"]].nunique()

,aeid,acid,asid,ccid,crtid
mcname,,,,,
週刊少年サンデー,2655,56,35,37,34
週刊少年ジャンプ,6393,104,63,64,64
週刊少年チャンピオン,433,20,12,13,14
週刊少年マガジン,1926,58,42,43,43


アニメ作品、アニメ各話、原作マンガ作品、原作マンガ作者の全てにおいて、`週刊少年ジャンプ`に関するものが最も多いことがわかります。

### `crtid`、`crtname`列の深掘り

`crtid`（マンガ作者ID）および`crtname`（マンガ作者名）について分析してみましょう。

In [44]:
# groupby(["crtid", "crtname"])で、これらの列の組み合わせごとにデータをグループ化
# .nunique()で、指定された列について各グループ内のユニークな値の数を計算
# .reset_index()で、結果のインデックスをリセットして、それを新しいデータフレームとして取得
df_crt_nunique = (
    df_ae_crt.groupby(["crtid", "crtname"])[["asid", "acid", "aeid", "date"]]
    .nunique()
    .reset_index()
)

# 計算したユニークな値の数について、'asid', 'acid', 'aeid', 'date'の各列の統計的な要約を取得
df_crt_nunique[["asid", "acid", "aeid", "date"]].describe()

,asid,acid,aeid,date
count,154.000000,154.000000,154.000000,154.000000
mean,1.168831,1.792208,82.467532,79.610390
std,0.454537,1.255968,121.545665,116.732436
min,1.000000,1.000000,11.000000,7.000000
25%,1.000000,1.000000,26.000000,26.000000
50%,1.000000,1.000000,49.000000,47.500000
75%,1.000000,2.000000,76.000000,74.750000
max,4.000000,8.000000,806.000000,782.000000


上記は、マンガ作者ごとのアニメ作品数（`acid`）、アニメ各話数（`aeid`）、そしてアニメ放送日数（`date`）の要約統計量を表したものです。
マンガ作者一人当たり、平均約1.79のアニメ作品が製作され、平均約82話が平均約79日間放送されていることがわかります。
ただし集計対象はアニメ化実績のあるマンガ作者に限定されていることにご注意ください。
つまり、四大少年誌に連載実績のあるマンガ作者に集計対象を広げると、上記の数値は全て下方修正されるはずです。

では、最も多くのアニメ作品が放送されたマンガ原作者は誰でしょうか？

In [45]:
# アニメ作品数で降順ソートし、上位5名を表示
df_crt_nunique.sort_values("acid", ascending=False).head()

,crtid,crtname,asid,acid,aeid,date
117,CCRT02724,秋本治,1,8,209,207
59,CCRT01512,小畑健,4,6,220,218
100,CCRT02446,満田拓也,1,6,160,159
4,CCRT00188,さとうふみや,2,6,258,244
47,CCRT01321,天樹征丸,2,6,258,244


最も多くのアニメ作品の原作者となったのは`秋本治`さんであることがわかりました。その内訳を見てみましょう。

In [46]:
# crtnameが秋本治と一致するレコードのうち、acnameごとに先頭の行を抽出し、特定の列のみ表示
df_ae_crt[df_ae_crt["crtname"] == "秋本治"].groupby("acname")[
    ["n_ae", "first_date", "ccname"]
].first()

,n_ae,first_date,ccname
acname,,,
こちら葛*飾区 亀有公園前 派出所［2016年スペシャル］,1,2016-09-18,こちら葛飾区亀有公園前派出所
こちら葛*飾区亀有公園前派出所[10月SP],2,2008-10-26,こちら葛飾区亀有公園前派出所
こちら葛*飾区亀有公園前派出所[4月SP],2,2008-04-06,こちら葛飾区亀有公園前派出所
こちら葛飾区 亀有公園前 派出所 [スペシャル] 両津の 浅草リニューアル 大作戦!! ～あぁ 思い出の花やしき～,1,2006-09-24,こちら葛飾区亀有公園前派出所
こちら葛飾区 亀有公園前 派出所 [スペシャル] 走れ! 両津式チンチン電車 ～思い出の大次郎号～,1,2006-04-02,こちら葛飾区亀有公園前派出所
こちら葛飾区亀有公園前派出所,198,1999-12-05,こちら葛飾区亀有公園前派出所
こちら葛飾区亀有公園前派出所 シートン探検隊!隅田川の誓い～思い出の白い鯨を探せ!～,2,2007-08-05,こちら葛飾区亀有公園前派出所
こちら葛飾区亀有公園前派出所 両さんの寿司食いねぇ!～頂上マグロ対決!!～,2,2007-09-30,こちら葛飾区亀有公園前派出所


全て`こちら葛飾区亀有公園前派出所`を原作としたアニメ作品であることがわかります。
では、アニメシリーズ（`asid`）で集計するとどうでしょうか？

In [47]:
# アニメシリーズ数で降順ソートし、上位5名を表示
df_crt_nunique.sort_values("asid", ascending=False).head()

,crtid,crtname,asid,acid,aeid,date
59,CCRT01512,小畑健,4,6,220,218
102,CCRT02471,瀬尾公治,3,3,50,50
29,CCRT00957,冨樫義博,3,4,336,336
84,CCRT02215,椎名高志,2,2,97,97
139,CCRT03016,赤松健,2,2,50,50


最も多くのアニメシリーズの原作者となったのは、`小畑健`さんであることがわかりました。
内訳を見てみましょう。

In [48]:
# crtnameが小畑健と一致するレコードのうち、asidごとに先頭の行を抽出し、特定の列のみ表示
df_ae_crt[df_ae_crt["crtname"] == "小畑健"].groupby("asid")[
    ["acname", "n_ae", "first_date", "ccname"]
].first()

,acname,n_ae,first_date,ccname
asid,,,,
C2285,人形[からくり]草紙 あやつり 左近,26,1999-08-16,人形草子あやつり左近
C2531,ヒカルの碁,76,2001-10-10,ヒカルの碁
C3898,DEATH NOTE,39,2006-10-04,DEATH NOTE
C4435,バクマン。[第1期],27,2010-10-02,バクマン。


`DEATH NOTE`、`バクマン。`、`ヒカルの碁`、そして`人形[からくり]草紙 あやつり 左近`と合計4シリーズをアニメ化されています。

筆者の個人的な予想では、`高橋留美子`さん原作のアニメシリーズが多いと思っていたのですが…。データを確認してみましょう。

In [49]:
# crtnameが高橋留美子と一致するレコードのうち、asidごとに先頭の行を抽出し、特定の列のみ表示
df_ae_crt[df_ae_crt["crtname"] == "高橋留美子"].groupby("asid")[
    ["acname", "n_ae", "first_date", "ccname"]
].first()

,acname,n_ae,first_date,ccname
asid,,,,
C2413,犬夜叉,171,2000-10-16,犬夜叉
C5983,境界の RINNE,26,2015-03-22,境界のRINNE


わずか2シリーズにとどまっていました。上記に加えて`高橋留美子`さん原作のアニメシリーズとして少なくとも：
- `うる星やつら`
- `めぞん一刻`
- `らんま1/2`

があると記憶していますので、データが欠損しています。

[アニメデータの基礎分析](../04/an_eda.ipynb)でも触れましたが、元データである[MADB Lab v1.0](https://github.com/mediaarts-db/dataset/tree/1.0)においてアニメデータ[^json]の放送日情報の欠損が多く見られます。
[折れ線グラフ](../08/line.ipynb)等で日付に関する情報が必須であることから、本書では[前処理](../appendix/an_preprocess.ipynb)として **日付情報のないアニメ作品・アニメ各話のデータを事前に除外** しています。
これにより`an_ae.csv`から上記の作品が除外され、それを元に作成された`mix_ae_crt.csv`にも影響が及んだという仮説が考えられます。
必須情報（`date`、`acname`）の欠損を理由に除外したレコードは`../../data/an/interim/ae_dropped.csv`に一覧化してありますので、確認してみましょう。

[^json]: 厳密には、[MADB Lab v1.0](https://github.com/mediaarts-db/dataset/tree/1.0)の`metadata_an-item_an201_json/*`において、`datePublished`が欠損しているレコードが多く見られます。

In [50]:
# grepコマンドで、うる星やつらを含む文字列をae_dropped.csvから抽出
!grep うる星やつら ../../data/an/interim/ae_dropped.csv

M22149,,,,C7956,うる星やつら,C5678


In [51]:
# grepコマンドで、めぞん一刻を含む文字列をae_dropped.csvから抽出
!grep めぞん一刻 ../../data/an/interim/ae_dropped.csv

M22488,,,,C8305,めぞん一刻,C6857


In [52]:
# grepコマンドで、らんま1/2を含む文字列をae_dropped.csvから抽出
!grep らんま1/2 ../../data/an/interim/ae_dropped.csv

M23689,,,,C8055,らんま1/2,C7122
M23753,"・第1話/激突! 出前格闘レース（1989/10/20）,・第2話/やっぱり猫が嫌い?（1989/11/03）,・第3話/私が女傑族のおばば!（1989/11/10）,・第4話/出た! 必殺天津甘栗拳!!（1989/11/17）,・第5話/白鳥拳の男ムース登場!（1989/11/24）,・第6話/爆走! 雪だるま運びレース（1989/12/01）,・第7話/さらわれたPちゃん!（1989/12/08）,・第8話/危機一髪! 死霊の盆踊り（1989/12/15）<B",,,C8595,らんま1/2 熱闘編,C7122


どうやら全て欠損値を理由に前処理段階で除外されてしまったようです。

## `mix_ce_ac.csv`の基礎分析

`mix_ce_ac.csv`は、マンガ各話とアニメ作品に関する情報を一つに集約したファイルです。
次のような列を持ちます。

- `ceid`：Comic Episode ID。マンガ各話ID
- `cename`：Comic Episode NAME。マンガ各話名
- `ccid`：Comic Collection ID。マンガ作品ID
- `ccname`：Comic Collection NAME。マンガ作品名
- `asid`：Anime Series ID。アニメシリーズID
- `miid`：Magazine Item ID。マンガ雑誌巻号ID
- `page_start`：当該マンガ雑誌巻号内での開始ページ
- `page_end`：当該マンガ雑誌巻号内での終了パージ
- `pages`：当該マンガ雑誌巻号内での合計ページ数
- `page_start_position`：当該マンガ雑誌巻号内での開始位置
- `two_colored`：当該マンガ各話が2色カラーページを含むか否か
- `four_colored`：当該マンガ各話が4色カラーページを含むか否か
- `date`：当該マンガ雑誌巻号の発売日
- `n_ce`：当該マンガ作品の合計各話数
- `n_2c`：当該マンガ作品の2色カラーの合計各話数
- `n_4c`：当該マンガ作品の4色カラーの合計各話数
- `first_date_cc`：当該マンガ作品の最初の掲載日
- `last_date_cc`：当該マンガ作品の最後の掲載日
- `mcid`：Magazine Collection ID。マンガ雑誌ID
- `mcname`：Magazine Collection NAME。マンガ雑誌名
- `acid`：Anime Collection ID。アニメ作品ID
- `acname`：Anime Collection NAME。アニメ作品名
- `n_ae`：当該アニメ作品の合計各話数
- `first_date_ac`：当該アニメ作品の最初の放送日
- `last_date_ac`：当該アニメ作品の最後の放送日

In [53]:
# pandasのread_csv関数でCSVファイルの読み込み
df_ce_ac = pd.read_csv(DIR_IN / FN_CE_AC)

### 全体像の把握

`df_ce_ac`を概観してみましょう。

In [54]:
# df_ce_acデータフレームの先頭5行を表示
# headメソッドはデータの概観を確認するのに便利
df_ce_ac.head().T

,0,1,2,3,4
ceid,CE00000,CE00026,CE00062,CE00086,CE00112
cename,第238話/この世代,第237話/トーナメント,第236話/絆,第235話/指先から…,第234話/何にも出来ないワケじゃない
ccid,C90829,C90829,C90829,C90829,C90829
miid,M535428,M535429,M535430,M535431,M535432
page_start,10.0,125.0,223.0,183.0,221.0
page_end,31.0,144.0,242.0,204.0,240.0
pages,22.0,20.0,20.0,22.0,20.0
page_start_position,0.021368,0.23855,0.478541,0.405765,0.472222
two_colored,False,False,False,False,False
four_colored,True,False,False,False,False


形状を確認してみましょう。

In [55]:
# df_ce_acデータフレームの形状（行数・列数）を取得
# shape属性を使用すると、(行数, 列数)の形で結果が返される
df_ce_ac.shape

(181681, 25)

欠損値の合計数と、全体に占める割合を確認してみましょう。

In [56]:
# df_ce_acデータフレーム内の欠損値（NaN）の情報を集計
# isna()メソッドを使用して欠損値の場所をTrueとして特定
# その後、agg()メソッドを使用して、各列における欠損値の合計と平均を計算し転置して表示
df_ce_ac.isna().agg([sum, "mean"]).T

,sum,mean
ceid,0.0,0.000000
cename,27536.0,0.151562
ccid,0.0,0.000000
miid,0.0,0.000000
page_start,0.0,0.000000
page_end,0.0,0.000000
pages,0.0,0.000000
page_start_position,0.0,0.000000
two_colored,0.0,0.000000
four_colored,0.0,0.000000


非常に多くの欠損があることがわかります。
- `cename`：約15%
- `acid`：約76%
- `acname`：約76%
- `asid`：約76%
- `n_ae`：約76%
- `first_date_ac`：約76%
- `last_date_ac`：約76%

特にアニメ作品に関する情報が欠損しているのは、このデータの作成過程に理由があります。

`mix_ce_ac`は、四大少年誌の **すべての** マンガ各話を対象にアニメ作品を対応づけたものです。
つまり、本データ中には **アニメ化されていないマンガ各話も含まれて** おり、それが欠損値を生んでいます。

要約統計量を確認してみましょう。

In [57]:
# df_ce_acデータフレームの記述統計情報を取得
# describe()メソッドを使用して、各列の中央値、平均、標準偏差などの基本的な統計量を転置表示
df_ce_ac.describe().T

,count,mean,std,min,25%,50%,75%,max
page_start,181681.0,209.882924,123.920451,1.000000,107.000000,203.000000,305.000000,581.0
page_end,181681.0,227.382651,122.137365,1.000000,125.000000,221.000000,322.000000,600.0
pages,181681.0,18.499728,7.712246,1.000000,17.000000,19.000000,20.000000,487.0
page_start_position,181681.0,0.514276,0.283281,0.002045,0.273529,0.519737,0.759524,1.0
n_ce,181681.0,218.286887,265.554237,1.000000,55.000000,152.000000,285.000000,1968.0
n_2c,181681.0,6.906451,15.956542,0.000000,0.000000,0.000000,6.000000,161.0
n_4c,181681.0,16.014608,26.655401,0.000000,1.000000,6.000000,19.000000,200.0
n_ae,43524.0,98.339008,156.534637,1.000000,25.000000,46.000000,97.000000,783.0


ユニークな値の数を確認してみましょう。

In [58]:
# df_ce_acデータフレームの各列に対してユニークな値の数をカウント
# nunique()メソッドを使用して、各列のユニークな値の数を計算
# その後、結果を新しいデータフレームとして整形し、列名を"nunique"とする
df_ce_ac.nunique().reset_index(name="nunique")

,index,nunique
0,ceid,181681
1,cename,151204
2,ccid,6953
3,miid,9822
4,page_start,550
5,page_end,568
6,pages,123
7,page_start_position,45726
8,two_colored,2
9,four_colored,2


`ceid`、すなわちマンガ各話IDを主キーとするデータであることがわかります。
以降、少し踏み込んだ分析を行います。

### `date`列の深堀り

`date`（掲載日）について分析してみましょう。

In [59]:
df_ce_ac["year"] = pd.to_datetime(df_ce_ac["date"]).dt.year

In [60]:
# 'acname'列の欠損値（NaN）の有無に基づいて、新しい列'is_animated'を作成
# 'acname'列に値があればTrue（アニメーションあり）、欠損値ならFalse（アニメーションなし）
df_ce_ac["is_animated"] = ~df_ce_ac["acname"].isna()

# date列から年情報を抜き出し、year列として格納
df_ce_ac["year"] = pd.to_datetime(df_ce_ac["date"]).dt.year
# year列を5年ごとに集計するためのyears列を追加
unit_years = 5
df_ce_ac["years"] = df_ce_ac["year"] // unit_years * unit_years
# yearsごとにis_animatedの平均値を算出
df_ce_ac.groupby("years")["is_animated"].mean().reset_index()

,years,is_animated
0,1970.0,0.053436
1,1975.0,0.070777
2,1980.0,0.073442
3,1985.0,0.099111
4,1990.0,0.189754
5,1995.0,0.280905
6,2000.0,0.340570
7,2005.0,0.362271
8,2010.0,0.404778
9,2015.0,0.323192


上記は、マンガ雑誌掲載年代（5年刻み）ごとに、アニメ化実績のあるマンガ作品の各話の割合を集計したものです。
1970-1974年に掲載されていたマンガ各話のうち、アニメ化実績のあるマンガ作品に紐づくものは約5%でしたが、徐々に上昇し、2010-2015年には約40%に到達しています。

ただし、この`is_animated`は **過去・現在・未来のどこかの時点でアニメ化されたマンガ作品の各話である** ことを表しており、 **当該マンガ各話が掲載時点でアニメ化されているわけではない** ことに注意しましょう。
例えば、2010-2015年に掲載されたマンガ作品の約39%が **その当時** アニメとして放送されていたわけではありません。

また、「未来」に関するアニメデータが限られているため、集計年が新しくなるほど`is_animeted`の割合が低くなる傾向があることにも注意が必要です。

### `ceid`、`cename`列の深掘り

`ceid`（マンガ各話ID）および`cename`（マンガ各話名）について分析してみましょう。

In [61]:
# 'is_animated'列（アニメーションがあるかないか）に基づいてデータをグループ化し、
# 'ceid'のユニークな値の数をそれぞれのグループで計算
# .reset_index()で、結果のインデックスをリセットして、それを新しいデータフレームとして取得
df_ce_ac.groupby("is_animated")["ceid"].nunique().reset_index()

,is_animated,ceid
0,False,138157
1,True,43524


上記は、アニメ化有無（`is_animated`）ごとに合計各話数（`ceid`）を集計したものです。
アニメ化されたマンガ作品の合計各話数は43524、そうでないマンガ作品の合計各話数は138157であることがわかりました。

では、マンガ雑誌別の傾向を見てみましょう。

In [62]:
# df_ce_acデータフレームを'mcname'に基づいてグループ化し、'is_animated'列に対して集約関数を適用
# .reset_index()で、結果のインデックスをリセットして、それを新しいデータフレームとして取得
df_ce_ac.groupby("mcname")["is_animated"].agg(["count", "sum", "mean"]).reset_index()

,mcname,count,sum,mean
0,週刊少年サンデー,46491,10240,0.220258
1,週刊少年ジャンプ,43371,18167,0.418874
2,週刊少年チャンピオン,45443,3224,0.070946
3,週刊少年マガジン,46376,11893,0.256447


上記は、マンガ雑誌別の合計各話数（`count`）と、うちアニメ化されたマンガ作品の各話数（`sum`）と、その割合（`mean`）を表しています。
本書で扱うデータにおいて、マンガ雑誌別の合計各話数、つまり集計期間中に雑誌に掲載された全てのマンガのエピソードの総数に大差はありません。
しかし、アニメ化されたマンガ作品か否かという観点で集計すると、`週刊少年ジャンプ`が最も多く全体の約42%のマンガ各話が該当しています。

ただし、この`is_animated`は **過去・現在・未来にどこかでアニメ化されたマンガ作品の各話である** ことを表しており、 **当該マンガ各話が掲載時点でアニメ化されているわけではない** ことに注意しましょう。
具体的には、`週刊少年ジャンプ`の約42%の解釈は「雑誌に掲載されているマンガ作品の約42%がアニメ化されている」ではなく、「過去・現在・未来のどこかでアニメ化されたマンガ作品の合計各話数が、雑誌の全各話数に対して約42%を占める」に過ぎないということです。
両者には大きな違いがあります。

### `ccid`、`ccname`列の深掘り

`ccid`（マンガ作品ID）および`ccname`（マンガ作品名）について分析してみましょう。

In [63]:
# 'is_animated'列（アニメーションがあるかないか）に基づいてデータをグループ化し、
# 'ccid'のユニークな値の数をそれぞれのグループで計算
# .reset_index()で、結果のインデックスをリセットして、それを新しいデータフレームとして取得
df_ce_ac.groupby("is_animated")["ccid"].nunique().reset_index()

,is_animated,ccid
0,False,6796
1,True,157


上記は、アニメ化有無（`is_animated`）ごとに合計マンガ作品数（`ccid`）を集計したものです。
四大少年誌においても、アニメ化されたマンガ作品は非常に稀であることがわかります。

では、マンガ雑誌別の傾向はどうなるでしょうか？

In [64]:
# df_ce_acデータフレームから'ccid'列を基準に重複を削除し
# 新しいインデックスを割り当てて新しいデータフレームdf_tmpを作成
df_tmp = df_ce_ac.drop_duplicates(subset=["ccid"], ignore_index=True)

# df_tmpデータフレームを'mcname'に基づいてグループ化し、
# 'is_animated'列に対して複数の集約関数（カウント、合計、平均）を適用
# .reset_index()で、結果のインデックスをリセットして、それをさらに扱いやすい形に整形
df_tmp.groupby("mcname")["is_animated"].agg(["count", "sum", "mean"]).reset_index()

,mcname,count,sum,mean
0,週刊少年サンデー,1496,37,0.024733
1,週刊少年ジャンプ,2129,64,0.030061
2,週刊少年チャンピオン,1736,13,0.007488
3,週刊少年マガジン,1592,43,0.027010


上記は、マンガ雑誌（`mcname`）ごとの合計マンガ作品数（`count`）、うちアニメ化された作品数（`sum`）、そしてその割合（`mean`）を整理したものです。
**マンガ各話** を基準とした際は全体の約42%がアニメ化と関連していた`週刊少年ジャンプ`ですら、 **マンガ作品** を基準に集計すると約3%まで落ち込みます。
これは、マンガ各話数が多いマンガ作品、つまり長期連載の人気作品ほどアニメ化する機会が多いことが原因と考えられます。
逆に言うと、`週刊少年ジャンプ`の **全各話の約42%は、わずか3%のマンガ作品で構成されている** ということです。

このあたりの議論は[メディア展開データの内訳を見る](./props.ipynb)で取り扱います。

### `miid`列の深掘り

`miid`（マンガ雑誌巻号ID）について分析してみましょう。

In [65]:
# df_ce_acデータフレームから'miid'と'ccname'の組み合わせで重複を削除し
# 新しいインデックスを割り当てて新しいデータフレームdf_tmpを作成
# （特別企画等で同一マンガ作品が複数回掲載されているパターンがあるため）
df_tmp = df_ce_ac.drop_duplicates(subset=["miid", "ccname"], ignore_index=True)

# df_tmpデータフレームを'mcname'と'miid'に基づいてグループ化し
# 'is_animated'列に対して合計と平均を計算
# .reset_index()で、結果のインデックスをリセット
df_mi_animated = (
    df_tmp.groupby(["mcname", "miid"])["is_animated"]
    .agg(["count", "sum", "mean"])
    .reset_index()
)

# 内容を表示
df_mi_animated.tail()

,mcname,miid,count,sum,mean
10107,週刊少年マガジン,M702008,22,6,0.272727
10108,週刊少年マガジン,M702009,24,7,0.291667
10109,週刊少年マガジン,M702010,23,5,0.217391
10110,週刊少年マガジン,M702011,27,9,0.333333
10111,週刊少年マガジン,M702012,24,7,0.291667


`df_mi_animated`は、マンガ雑誌巻号（`miid`）ごとに、マンガ作品数（`count`）、そのうちアニメ化された作品数（`sum`）、そしてその割合（`mean`）を格納した`DataFrame`です。
ややこしいですが、マンガ雑誌全体（`mcname`）に対する集計結果ではなく、特定の週に発売された **雑誌巻号** 内での集計であることに注意してください。
例えば一行目は、`週刊少年マガジン`の`M702008`号には合計22作品掲載されましたが、うちアニメ化実績のある作品が6作品あり、その割合は約$0.27 \risingdotseq \frac{6}{22}$であるということを表しています。

`df_mi_animated`の要約統計量を取ることで、全体の傾向を見てみましょう。

In [66]:
# sum、mean列について要約統計量を算出
df_mi_animated[["sum", "mean"]].describe()

,sum,mean
count,10112.000000,10112.000000
mean,4.255538,0.212150
std,4.427726,0.211994
min,0.000000,0.000000
25%,0.000000,0.000000
50%,2.000000,0.130435
75%,8.000000,0.375000
max,16.000000,1.000000


全マンガ雑誌巻号において、平均約4作品がアニメ化実績があり、その割合の平均は約0.21であることがわかりました。
では、最も多くのアニメ化作品が掲載された雑誌巻号を特定してみましょう。

In [67]:
# アニメ化作品数の順に降順ソートし、上位5巻号を表示
df_mi_animated.sort_values(["sum", "mean", "miid"], ascending=False).head()

,mcname,miid,count,sum,mean
4756,週刊少年ジャンプ,M616464,21,16,0.761905
2725,週刊少年ジャンプ,M542873,21,16,0.761905
2724,週刊少年ジャンプ,M542872,21,16,0.761905
2723,週刊少年ジャンプ,M542871,21,16,0.761905
2722,週刊少年ジャンプ,M542870,21,16,0.761905


最も多くのアニメ化作品を掲載していた巻号の一つは、`週刊少年ジャンプ`の`M616464`であることがわかりました。
雑誌に掲載されていた21作品のうち、76%以上を占める16作品にアニメ化実績がありました。
内訳を見てみましょう。

In [68]:
# miidがM616464と一致し、かつアニメ化実績があるマンガ作品名一覧を取得
df_ce_ac[(df_ce_ac["miid"] == "M616464") & df_ce_ac["is_animated"]]["ccname"].unique()

array(['こちら葛飾区亀有公園前派出所', 'ONE PIECE', 'トリコ', 'NARUTO-ナルト-', 'SKET DANCE',
       '銀魂', 'べるぜバブ', 'めだかボックス', 'BLEACH', '黒子のバスケ', 'ハイキュー!!', 'ニセコイ',
       '斉木楠雄のΨ難', '暗殺教室', '食戟のソーマ', 'ワールドトリガー'], dtype=object)

**雑誌掲載当時アニメ化されていたとは限りません** が、確かにアニメ化実績のある顔ぶれです。
読者としてはもちろん嬉しいですが、もし自分が新連載を控える新人マンガ家だったらと考えると複雑な気持ちになります[^opinion]。

[^opinion]: このような過酷な状況で戦い続ける全てのマンガ作者を、筆者は心から尊敬しています。

### `page_start`、`page_end`、`pages`、`page_start_position`列の深掘り

`page_start`（開始ページ）、`page_end`（終了ページ）、`pages`（ページ数）、そして`page_start_position`（雑誌巻号中の掲載位置）について分析してみましょう。
`page_start_position`が小さいほど、雑誌巻頭よりに掲載されていたことを表します。

In [69]:
# is_animated別に各指標の平均値を算出
df_ce_ac.groupby("is_animated")[
    ["page_start", "page_end", "pages", "page_start_position"]
].mean()

,page_start,page_end,pages,page_start_position
is_animated,,,,
False,217.147238,234.696809,18.549570,0.544581
True,186.824017,204.165529,18.341513,0.418081


上記は、アニメ化実績の有無（`is_animated`）に対するページ関連の各指標（`page_start`、`page_end`、`pages`、`page_start_position`）に関する平均値を算出したものです。
平均的にアニメ化実績のあるマンガ作品（`is_animated`が`True`）のほうが、雑誌巻頭よりに掲載されているように見えます。

では次は、特に`page_start_position`に着目し、マンガ雑誌別に集計してみましょう。

In [70]:
# df_ce_acデータフレームでピボットテーブルを作成
# 'is_animated'をインデックスとして、'mcname'を列とし、
# 'page_start_position'の値に基づく平均値を集約関数として計算
df_ce_ac.pivot_table(
    index="is_animated", columns="mcname", values="page_start_position", aggfunc="mean"
)

mcname,週刊少年サンデー,週刊少年ジャンプ,週刊少年チャンピオン,週刊少年マガジン
is_animated,,,,
False,0.539659,0.572595,0.530063,0.547054
True,0.389284,0.433425,0.285667,0.455331


全てのマンガ雑誌において、アニメ化実績のあるマンガ作品はそうでないマンガ作品より平均的に巻頭よりに掲載されているように見えます。
特に`週刊少年チャンピオン`においてその差が大きいように見えますが、他誌と比較してアニメ化実績のあるマンガ作品が多くないため解釈には注意が必要です。

### `two_colored`、`four_colored`列の深堀り

`two_colored`（二色カラーページを含む各話か否か）および`four_colored`（四色カラーページを含む各話か否か）について分析してみましょう。

In [71]:
# is_animated別に各指標の平均値を算出
df_ce_ac.groupby("is_animated")[["two_colored", "four_colored"]].mean()

,two_colored,four_colored
is_animated,,
False,0.044493,0.065281
True,0.019369,0.093787


上記は、アニメ化実績の有無（`is_animated`）ごとに、二色カラー獲得率（`two_colored`）と四色カラー獲得率（`four_colored`）を整理したものです。
想定通り、`four_colored`に関してはアニメ化実績のあるマンガ各話（`is_animated`が`True`）のほうが高くなりました。
一方で、`two_colored`に関しては逆の傾向を示しました。
何が起きているのでしょうか？

In [72]:
# df_ce_acデータフレームを'ccid'と'ccname'に基づいてグループ化し
# 'two_colored', 'four_colored', 'is_animated', 'n_ce'の各列について平均を計算
# （'is_animated'と'n_ce'に関してはccid内で同じ値を取るため平均をとっても問題ない）
df_cc_colored = (
    df_ce_ac.groupby(["ccid", "ccname"])[
        ["two_colored", "four_colored", "is_animated", "n_ce"]
    ]
    .mean()
    .reset_index()
)

# df_cc_coloredデータフレームを'two_colored'列の値で降順に並び替え、上位5行を表示
df_cc_colored.sort_values("two_colored", ascending=False).head(5)

,ccid,ccname,two_colored,four_colored,is_animated,n_ce
5333,C93922,流血鬼,1.0,0.0,0.0,1.0
3079,C90621,荒野への帰還,1.0,0.0,0.0,1.0
3945,C91841,南野陽子物語,1.0,0.0,0.0,1.0
4633,C92894,コンクリート・ライブ,1.0,0.0,0.0,1.0
2006,C89268,ニートに翔んで,1.0,0.0,0.0,1.0


上記は、二色カラー獲得率（`two_colored`）に関してマンガ作品（`ccid`）を降順ソートし、上位5作品を表示したものです。
マンガ作品の合計話数（`n_ce`）が1話あるいは非常に少ない作品が多くが含まれていることがわかります。

では、`n_ce`が一定以上の作品に限って再集計してみましょう。

In [73]:
# n_ceの最小値を8として設定
min_nce = 8
# is_animated別に各指標の平均値を算出
df_ce_ac[df_ce_ac["n_ce"] >= min_nce].groupby("is_animated")[
    ["two_colored", "four_colored"]
].mean()

,two_colored,four_colored
is_animated,,
False,0.042763,0.060272
True,0.019365,0.093882


上記は、合計各話数が8話以上のマンガ作品に絞って、先ほどと同様にアニメ化実績有無（`is_animated`）と二色カラー獲得率（`two_colored`）と四色カラー獲得率（`four_colored`）の関係を整理したものです。
条件を変更しても、全体の傾向は変わりませんでした。他の部分に原因がありそうです。

では、`years`について注目してみましょう。

In [74]:
# 年代別に各指標の平均値を算出して表示
df_ce_ac.groupby("years")[["is_animated", "two_colored", "four_colored"]].mean()

,is_animated,two_colored,four_colored
years,,,
1970.0,0.053436,0.033272,0.094522
1975.0,0.070777,0.126122,0.065824
1980.0,0.073442,0.125217,0.042401
1985.0,0.099111,0.076740,0.042873
1990.0,0.189754,0.035601,0.040300
1995.0,0.280905,0.019133,0.048192
2000.0,0.340570,0.004701,0.067801
2005.0,0.362271,0.000563,0.092917
2010.0,0.404778,0.001373,0.111006


上記は、5年刻みの各年代（`years`）において、マンガ雑誌中に掲載された各話のうちアニメ化実績のあるもの（`is_animated`）、二色カラーを獲得したもの（`two_colored`）、そして四色カラーを獲得したもの（`four_colored`）の割合を集計したものです。
二色カラー獲得率が最も高まるのは1975-1984年頃の10年間であり、それ以降は減少の一途をたどります。
一方でアニメ化実績率が高まるのはその少し後の1990年代以降です。
二色カラー掲載が隆盛を極めた時代には、アニメ化によるメディア展開がそれほど多くなかったのかもしれません。

### `n_ce`列の深掘り

`n_ce`（合計各話数）について分析してみましょう。

In [75]:
# df_ce_acデータフレームから'ccid'列を基準に重複を削除し、
# 新しいインデックスを割り当てて新しいデータフレームdf_cc_acを作成
df_cc_ac = df_ce_ac.drop_duplicates(subset=["ccid"], ignore_index=True)

# df_cc_acデータフレームを'is_animated'に基づいてグループ化し、
# 'n_ce'の列についての要約統計量を計算
df_cc_ac.groupby("is_animated")["n_ce"].describe()

,count,mean,std,min,25%,50%,75%,max
is_animated,,,,,,,,
False,6796.0,20.142731,50.005947,1.0,1.0,1.0,16.0,635.0
True,157.0,275.012739,225.826516,1.0,161.0,241.0,337.0,1968.0


上記は、マンガ作品ごとのアニメ化実績有無（`is_animated`）と合計各話数（`n_ce`）の要約統計量を整理したものです。
アニメ化実績のあるマンガ作品（`is_animated`が`True`）ほど、合計各話数が多い傾向があるように見えます。

### `n_2c`、`n_4c`列の深堀り

`n_2c`（マンガ作品の二色カラー獲得回数）および`n_4c`（マンガ作品の四色カラー獲得回数）について分析してみましょう。

In [76]:
# df_cc_acデータフレームを'is_animated'に基づいてグループ化し、
# 'n_2c'、'n_4c'の列についての要約統計量を計算して転置表示
# （df_ce_acではなくdf_cc_acであることに注意！）
df_cc_ac.groupby("is_animated")[["n_2c", "n_4c"]].describe().T

is_animated        False       True 
n_2c count   6796.000000  157.000000
     mean       0.873896    5.331210
     std        4.380116   11.839952
     min        0.000000    0.000000
     25%        0.000000    0.000000
     50%        0.000000    0.000000
     75%        0.000000    4.000000
     max      161.000000   72.000000
n_4c count   6796.000000  157.000000
     mean       1.309005   25.643312
     std        4.730921   25.115378
     min        0.000000    0.000000
     25%        0.000000    9.000000
     50%        0.000000   19.000000
     75%        1.000000   35.000000
     max      200.000000  158.000000

上記は、マンガ作品ごとのアニメ化実績有無（`is_animated`）と、二色カラー獲得回数（`n_2c`）および四色カラー獲得回数（`n_4c`）の要約統計量を整理したものです。
全体傾向として、アニメ化実績のあるマンガ作品（`is_animeted`が`True`）ほど、二色カラー獲得回数についても、四色カラー獲得回数についても多くなる傾向が見られます。

In [77]:
# df_cc_acのうち、n_acに関して降順ソートし、特定の列について上位5行を表示
df_cc_ac.sort_values("n_2c", ascending=False)[
    ["mcname", "ccname", "n_ce", "n_2c", "n_4c", "is_animated"]
].head()

,mcname,ccname,n_ce,n_2c,n_4c,is_animated
3232,週刊少年チャンピオン,がきデカ,332,161,70,False
3184,週刊少年チャンピオン,ドカベン,635,109,200,False
3223,週刊少年チャンピオン,らんぽう,457,79,29,False
446,週刊少年マガジン,釣りキチ三平,482,74,17,False
1333,週刊少年ジャンプ,キン肉マン,400,72,22,True


上記は、四代少年誌に掲載実績のあるマンガ作品のうち、二色カラー獲得回数が多かった上位5作品をまとめた表です。
特に`週刊少年チャンピオン`のマンガ作品が目立ちます。

ここでも違和感を感じる方がいるかもしれません。
`is_animated`が`False`となっている`がきデカ`も`ドカベン`も`らんぽう`も`釣りキチ三平`も、アニメ化された実績のあるマンガ作品です。
MADB Lab v1.0のアニメ各話の元データ（`metadata_an-item_an201_json\metadata_an-item_an201_0000*.json`）を調査したところ、前述した`高橋留美子`さん作品と同様に`datePublished`が欠損した状態で存在することがわかりました。

In [78]:
# grepコマンドで、がきデカを含む文字列をae_dropped.csvから抽出
!grep がきデカ ../../data/an/interim/ae_dropped.csv

M23737,,,,C8579,がきデカ,C7146


### `first_date_cc`、`last_date_cc`列の深掘り

`first_date_cc`（マンガ作品の最初の掲載日）および`last_date_cc`（マンガ作品の最後の掲載日）について分析してみましょう。

In [79]:
# 'first_date_cc'列と'last_date_cc'列のデータを日付型に変換
df_cc_ac["first_date_cc"] = pd.to_datetime(df_cc_ac["first_date_cc"])
df_cc_ac["last_date_cc"] = pd.to_datetime(df_cc_ac["last_date_cc"])

# 'is_animated'でデータをグループ化し、'first_date_cc'（最初の日付）と'last_date_cc'（最後の日付）の
# 統計的な要約（count, mean, std, min, 25%, 50%, 75%, max）を計算
# .T（転置）を用いて、結果の表示形式を行と列を入れ替えて、より読みやすく整理
df_cc_ac.groupby("is_animated")[["first_date_cc", "last_date_cc"]].describe().T

is_animated                                  False  \
first_date_cc count                           6796   
              mean   1993-09-05 08:51:37.822248320   
              min              1970-07-27 00:00:00   
              25%              1980-08-24 00:00:00   
              50%              1993-03-28 00:00:00   
              75%              2006-12-14 00:00:00   
              max              2017-07-31 00:00:00   
last_date_cc  count                           6796   
              mean   1994-04-14 15:13:27.298410880   
              min              1970-07-27 00:00:00   
              25%              1981-01-12 00:00:00   
              50%              1994-01-01 00:00:00   
              75%              2007-11-20 18:00:00   
              max              2017-07-31 00:00:00   

is_animated                                  True   
first_date_cc count                            157  
              mean   1998-12-17 00:36:41.273885312  
              min              1970-08-02 00:00:00  
              25%              1993-06-14 00:00:00  
              50%              2001-06-06 00:00:00  
              75%              2007-08-09 00:00:00  
              max              2016-05-23 00:00:00  
last_date_cc  count                            157  
              mean   2007-03-09 00:09:10.318471424  
              min              1971-01-17 00:00:00  
              25%              2002-05-01 00:00:00  
              50%              2009-09-14 00:00:00  
              75%              2016-08-31 00:00:00  
              max              2017-07-31 00:00:00

上記は、マンガ作品ごとのアニメ化実績の有無（`is_animated`）とマンガ作品の最初と最後の掲載日（`first_date_cc`、`last_date_cc`）の要約統計量を集計した表です。
全体傾向として、アニメ化実績のある（`is_animated`が`True`の）マンガ作品のほうが、掲載日が新しいように見えます。

### `mcid`、`mcname`列の深掘り

`mcid`（マンガ雑誌ID）および`mcname`（マンガ雑誌名）について分析してみましょう。

In [80]:
# アニメ化実績のあるマンガ作品に対して、mcnameごとの各列の平均値を算出
df_cc_ac[df_cc_ac["is_animated"]].groupby("mcname")[["n_ce", "n_2c", "n_4c"]].mean()

,n_ce,n_2c,n_4c
mcname,,,
週刊少年サンデー,276.756757,3.621622,20.216216
週刊少年ジャンプ,283.859375,10.265625,32.296875
週刊少年チャンピオン,248.000000,1.692308,34.615385
週刊少年マガジン,268.511628,0.558140,17.697674


上記は、アニメ化実績があるマンガ作品に対して、合計各話数（`n_ce`）、二色カラー獲得数（`n_2c`）、そして四色カラー獲得率（`n_4c`）の平均値をマンガ雑誌（`mcname`）に集計した表です。
雑誌によって違いがあるように見えますが、これが **雑誌自体の特徴なのか、アニメ化された作品の特徴なのか** 判別できません。

そこで、上記をアニメ化有無にかかわらないマンガ作品全体の集計値で割ることで、指標を相対化してみましょう。

In [81]:
# 'is_animated'がTrue場合のデータをフィルタリングして、'mcname'でグループ化
# 'n_ce'、'n_2c'、'n_4c'の平均値を計算
df_tmp_animated = (
    df_cc_ac[df_cc_ac["is_animated"]].groupby("mcname")[["n_ce", "n_2c", "n_4c"]].mean()
)

# 全コンテンツ（アニメーションの有無に関わらず）を'mcname'でグループ化し、
# 同じく'n_ce'、'n_2c'、'n_4c'の平均値を計算
df_tmp_all = df_cc_ac.groupby("mcname")[["n_ce", "n_2c", "n_4c"]].mean()

# アニメーションがあるコンテンツの平均値を、全コンテンツの平均値で割る
# これにより、アニメーションがあるコンテンツの特性が全体と比べてどのように異なるかを数値的に把握できる
df_tmp_animated / df_tmp_all

,n_ce,n_2c,n_4c
mcname,,,
週刊少年サンデー,9.024349,3.094201,12.190028
週刊少年ジャンプ,14.023358,9.256889,15.821456
週刊少年チャンピオン,9.474022,2.002622,17.130076
週刊少年マガジン,9.364086,0.742321,10.890877


上記は、アニメ化実績のあるマンガ作品の各指標の雑誌別の平均値を、全マンガ作品に対する同様の平均値で割った結果を示した表です。
表の見方の例としては：
- `週刊少年サンデー`でアニメ化実績のあるマンガ作品の合計各話数（`n_ce`）の平均値は、同誌で掲載された全マンガ作品の平均と比較して約8.76倍も多い
- `週刊少年マガジン`でアニメ化実績のあるマンガ作品の四色カラー獲得回数（`n_4c`）の平均値は、同誌で掲載された全マンガ作品の平均と比較して約10.9倍も多い

です。

全体を通して、全ての指標が$1$を超えています。
つまり、全てのマンガ雑誌において、アニメ化実績のあるマンガ作品の各種指標値は平均値より大きいことがわかります。

ただし、この集計結果は **いかなる因果関係も示していない** ことに注意が必要です。
合計各話数やカラー獲得数が多いからアニメ化されるのか、それともアニメ化されたから合計各話数やカラー獲得数が多くなったのか、あるいはただの偶然かを結論づけることはできません。

また、何度か触れているようにアニメ化実績の有無（`is_animated`）に欠損が含まれることも、常に意識しておきましょう。

### `acid`、`acname`列の深掘り

`acid`（アニメ作品ID）および`acname`（アニメ作品名）について分析してみましょう。
ここでは特に、`acname`と類似度が高いが原作ではない`ccname`の存在について取り扱います。

In [82]:
# 'ccname_closest'列を作成し、各行に対して'find_closest_match'関数を適用
# この関数は、現在の行のデータに基づいてdf_cc_acデータフレーム内の最も近いマッチを見つける
# applyメソッドを使い、axis=1は行ごとの操作を意味し、追加の引数としてdf=df_cc_acを渡している
df_cc_ac["ccname_closest"] = df_cc_ac.apply(find_closest_match, axis=1, df=df_cc_ac)

# アニメーションがあるコンテンツ（'is_animated'がTrue）のみをフィルタリングし、特定の列を抽出
# reset_index(drop=True)でインデックスをリセットし、新しい連番のインデックスを作成することで、
# 選択したデータを新しいデータフレームdf_tmpとして整理
df_tmp = df_cc_ac[df_cc_ac["is_animated"]][
    ["acname", "ccname", "ccname_closest"]
].reset_index(drop=True)

`df_tmp`は、アニメ作品名（`acname`）ごとに、その原作マンガ作品名（`ccname`）と **最もアニメ作品名に近い文字列の** マンガ作品名（`ccname`）を格納した`DataFrame`です。

In [83]:
# ccname（原作マンガ作品名の正解）とccname_closest（acnameと最も近いccname）が
# 一致している割合を算出
(df_tmp["ccname"] == df_tmp["ccname_closest"]).mean()

0.8343949044585988

`ccname`（原作マンガ作品名の正解）と、`ccname_closed`（`acname`と最も近い文字列を含む`ccname`）の一致率は約84%でした。
つまり、現在手元にある`acname`[^acname]を前提に作品名の類似度を基準に原作マンガを探すと、約16%は誤ったペアになるということです。

[^acname]: 説明のため簡易化しましたが、メディア展開データの前処理の段階では **この** `acname`のリスト（四大少年誌を原作としたアニメ作品名のリスト）は存在しないので、 **全アニメ作品を対象に** 同様の処理を実行する必要がありました。

では、マッチングに失敗する組合せを確認してみましょう。

In [84]:
# ccnameとccname_closestが一致しない作品リスト
df_tmp[df_tmp["ccname"] != df_tmp["ccname_closest"]]

,acname,ccname,ccname_closest
4,CØDE: BREAKER,CODE:BREAKER コード:ブレイカー,SWORD BREAKER
7,はじめの 一歩 THE FIGHTING!,はじめの一歩,IN THE TRAIN
16,ツバサ・クロニクル 年代記[第1期],ツバサ ～RESERVoir CHRoNiCLE～,バイオハザード アンブレラ・クロニクルズ 崩壊への序曲
17,School Rumble,スクールランブル,Moon Walker
22,ゲットバッカーズ 奪還屋,Get Backers 奪還屋,ロボットボーイズ
25,RAVE GROOVE ADVENTURE,RAVE,SPACE ADVENTURE コブラ
28,アソボット戦記 五九,アソボット五九,アソボット戦記五九
36,巨人の星 【特別篇】 猛虎 花形満 ～『巨人の星』全182話より特別編集～,巨人の星,巨人の星外伝それからの飛雄馬
40,ドクタースランプ,Dr.スランプ,ドクター・ジュン
54,ハンター×ハンター,HUNTER×HUNTER,ハングリーハート


面白いので全て取り上げたいですが、紙幅の都合から代表的なものだけをピックアップします。
例えば、アニメ作品`CØDE: BREAKER`の原作として、副題のぶん類似度が下がってしまった`CODE:BREAKER コード:ブレイカー`より`SWORD BREAKER`を選出する気持ちはわかります。
アニメ作品`School Rumble`と原作マンガ作品`スクールランブル`のように言語が異なる場合は、単純な文字列の一致度を見るだけでは対応できません。

`mix_ce_ac.csv`において、アニメ作品`ガンバリスト!駿`と対応付けられているマンガ作品が`ガンバ!!Fly high`であることに違和感を覚える方もいるかもしれません。
正確な原作マンガ作品名は`!`が一つだけの`ガンバ! Fly high`ですが、本書で扱うデータセットには以下2種類が存在しています：
- `ガンバ!!Fly high`（`C92636`）：第1話から第304話までを収録
- `ガンバ! Fly high`（`C92633`）：第305話以降を収録

おそらく前者は`ccname`の誤植です。
しかし、本書では **掲載日付が早い方を原作する** というポリシーのもと紐づけを行っているため、処理の一貫性を保つために **マンガ作品名に誤植の疑いのある`ガンバ!!Fly high`** を採用しました。
詳細は[メディア展開データの前処理](../appendix/mix_preprocess.ipynb)を参照ください。

非常に難しいのは、アニメ作品`アソボット戦記 五九`と原作マンガ作品`アソボット五九`の対応付けです。
類似度が高いため問題なく対応できそうに見えますが、実際にデータを見てみましょう。

In [85]:
# ccnameに「アソボット」を含むデータを抽出し、特定の列を表示しつつ、dateで昇順ソート
df_ce_ac[df_ce_ac["ccname"].str.contains("アソボット")][
    ["date", "ccname", "cename", "acname", "first_date_ac"]
].sort_values("date")

,date,ccname,cename,acname,first_date_ac
18384,2002-01-17,アソボット五九,第1話 大悪党・五九 参上!!,アソボット戦記 五九,2002-10-01
18383,2002-01-23,アソボット五九,第2話 ハーモニカを持った少年,アソボット戦記 五九,2002-10-01
18382,2002-01-30,アソボット五九,第3話 ゴータ・マ村長の決意,アソボット戦記 五九,2002-10-01
18381,2002-02-06,アソボット五九,第4話 サルってゆうんじゃねえ!,アソボット戦記 五九,2002-10-01
18380,2002-02-13,アソボット五九,第5話 脅威!マリオネットシール,アソボット戦記 五九,2002-10-01
18379,2002-02-20,アソボット五九,第6話 ナミダの温度,アソボット戦記 五九,2002-10-01
18378,2002-02-27,アソボット五九,第7話 はるかなる旅路へ,アソボット戦記 五九,2002-10-01
18377,2002-03-06,アソボット五九,第8話 美少女盗賊・魅音サマ,アソボット戦記 五九,2002-10-01
18376,2002-03-13,アソボット五九,第9話 白い砂漠,アソボット戦記 五九,2002-10-01
18375,2002-03-20,アソボット五九,第10話 強敵・ヤズー現る!,アソボット戦記 五九,2002-10-01


時系列としては、`2002-01-17`からマンガ作品`アソボット五九`が`週刊少年マガジン`にて連載を開始し、`2002-10-01`にアニメ作品`アソボット戦記五九`が放送を開始[^media-mix]しました。
更に、それと連動する形で`2002-10-09`から2話`アソボット戦記五九`が掲載されています。
非常にややこしいですが、時間的な前後関係を加味すると、筆者はアニメ作品`アソボット戦記五九`の原作マンガは同名の`アソボット戦記五九`ではなく`アソボット五九`であるべきと考えます。

「たとえアニメ作品名とマンガ作品名が **完全に一致していても原作とは限らない** 」という、分析者にとって絶望的な事実が明らかにりました。

[^media-mix]: `アソボット戦記五九`はエイベックス主導で展開されたメディアミックス企画であるため、マンガとアニメが非常に近い時期に公開されています。本書では、時間的な前後関係や関係者の証言から **アニメが原作であると断定できる場合を除き、基本的にマンガが原作と判断** しています。

### `asid`列の深堀り

`asid`（アニメシリーズID）について分析してみましょう。

In [86]:
# asidでグループ化し、mcname、ccname、ccidに関しては先頭のものを抽出
# ceidに関してはユニーク数を集計
df_tmp = (
    df_ce_ac.groupby("asid")
    .agg({"mcname": "first", "ccname": "first", "ccid": "first", "ceid": "nunique"})
    .reset_index()
)
# ceidのユニーク数（つまり合計各話数）の要約統計量を算出
df_tmp["ceid"].describe().reset_index()

,index,ceid
0,count,152.000000
1,mean,286.342105
2,std,229.496717
3,min,16.000000
4,25%,166.250000
5,50%,244.000000
6,75%,341.250000
7,max,1968.000000


上記は、アニメシリーズに対応する原作マンガ作品の合計各話数の要約統計量を整理した表です。
アニメシリーズの原作となったマンガ作品は、平均で約286話、最小で16話、最大で1968話の合計各話数を持つことがわかりました。
例えば、合計各話数が小さい原作マンガ作品を抽出してみましょう。

In [87]:
# 合計各話数に関して昇順ソートし、上位5行を表示
df_tmp.sort_values("ceid").head(5)

,asid,mcname,ccname,ccid,ceid
111,C4571,週刊少年ジャンプ,レベルE,C89851,16
53,C2640,週刊少年マガジン,アソボット五九,C90306,27
22,C1994,週刊少年マガジン,中華一番!,C90794,30
33,C2285,週刊少年ジャンプ,人形草子あやつり左近,C88332,32
89,C3508,週刊少年ジャンプ,初恋限定。,C89375,32


アニメ化実績のある四大少年誌のマンガ作品のうち、合計各話数が最も小さいのは`レベルE`のようです。

### `n_ae`列の深掘り

`n_ae`（合計アニメ各話数）について分析してみましょう。
例えば、`n_ae`が多いアニメ作品を抽出してみます。

In [88]:
# n_aeに関して降順ソートしたうえで特定の列のみ選択し、上位5行を表示
df_cc_ac.sort_values("n_ae", ascending=False)[
    ["mcname", "ccname", "acname", "n_ae"]
].head()

,mcname,ccname,acname,n_ae
1347,週刊少年ジャンプ,ONE PIECE,ONE PIECE,783.0
4540,週刊少年サンデー,名探偵コナン,名探偵コナン,729.0
1359,週刊少年ジャンプ,BLEACH,BLEACH,369.0
1349,週刊少年ジャンプ,NARUTO-ナルト-,NARUTO,221.0
1352,週刊少年ジャンプ,銀魂,銀魂,201.0


四大少年誌に掲載されたマンガ作品を原作としたアニメ作品のうち、最も放映回数が多いものは`ONE PIECE`であることがわかりました。
こうしてまとめてみると、`週刊少年ジャンプ`原作のアニメ作品が多いことがわかります。

### `first_date_ac`、`last_date_ac`列の深掘り

`first_date_ac`（アニメ作品の最初の放送日）および`last_date_ac`（アニメ作品の最後の放送日）について分析してみましょう。
例えば、`first_date_ac`が古いアニメ作品を列挙してみます。

In [89]:
# first_date_acに関して昇順ソートしたうえで特定の列のみ選択し、上位5行を表示
df_cc_ac.sort_values("first_date_ac")[
    ["mcname", "ccname", "acname", "first_date_ac"]
].head(5)

,mcname,ccname,acname,first_date_ac
2225,週刊少年ジャンプ,CITY HUNTER シティ・ハンター,シティーハンター3,1990-01-14
2184,週刊少年ジャンプ,まじかる☆タルるートくん,まじかる★タルるートくん,1990-09-02
448,週刊少年マガジン,三つ目がとおる,三つ目がとおる,1990-10-18
5407,週刊少年サンデー,おれは直角,おれは直角,1991-01-05
1333,週刊少年ジャンプ,キン肉マン,キン肉マン キン肉星王位争奪編,1991-10-06


四大少年誌に掲載されたマンガ作品を原作としたアニメ作品のうち、最も古くからアニメ放送が開始されたのは`シティーハンター3`であることがわかります。
しかし、`acname`を見る限りこのアニメシリーズの最初の放送ではなさそうです。

In [90]:
# acnameにシティーを含む行に対して、acnameの値の数を集計し、reset_indexでDataFrame化
df_ce_ac[df_ce_ac["acname"].str.contains("シティー") > 0][
    "acname"
].value_counts().reset_index()

,acname,count
0,シティーハンター3,337


本書で扱うデータ上には、`シティーハンター3`の前に放送されたと想像される1期や2期のデータが存在しないことがわかりました。

## まとめ

ここでは、`mix_ae_crt.csv`そして`mix_ce_ac.csv`について基礎分析を行いました。
重要な発見や示唆がたくさんありますので、もう一度復習しておきましょう。

まず大前提として、本書で扱うアニメデータは、 [MADB Lab v1.0](https://github.com/mediaarts-db/dataset/tree/1.0)として公開されているものをGitHub経由で取得し、[所定の前処理](../appendix/mix_preprocess.ipynb)を行ったものです。
マンガデータに関しては、発行日が 
**1970年7月27日から2017年7月31日まで**
の
**四大少年誌（週刊少年サンデー、週刊少年ジャンプ、週刊少年チャンピオン、そして週刊少年マガジン）**
を対象としています。
また、アニメデータに関しては、 放送日が
**1963年1月1日から2017年10月15日**
までの
**テレビレギュラー放送**
を対象としています。 
肝心のマンガデータとアニメデータの紐づけは、
**筆者のドメイン知識をベースに基本的に手作業で実施**
しました。
過不足や誤りがある可能性は否定できません。

`mix_ae_crt.csv`は、アニメ各話とその原作マンガの作者についてまとめたファイルです。
今後分析で用いる際は、以下に特に注意するべきでしょう：

- `aename`と`aeno`にわずかに欠損がある
- `aeid`と`crtid`の組合せが主キーとなっているため、それぞれ単独では重複がある
- `date`に関しては、`1990-01-14`移行のアニメ作品のみを対象としており、それ以前のアニメ作品が欠損している
    - 理由の一つとして、元データで放送日に関する情報が欠損しているこいとが考えられる（例：`うる星やつら`、`らんま1/2`、`シティーハンター`、`がきデカ`、`ドカベン`、等々）
- `n_ae`の数え方はアニメ作品に依存する。1放送枠ごとに数える場合もあるし、1話ごとに数える場合もある。ギャグアニメのように1放送枠に複数話が放送される作品の場合、集計結果に差が出ることがある。
- `aename`に複数話のサブタイトルをまとめて格納しているパターンがある
    - 例：`もっと ToLOVEる`の`もっとトラブル01\u3000もう一度ここから（10/6）\u3000もっとトラブル02\u3000お風呂場戦争（10/6）\u3000もっとトラブル03\u3000チクタク チクタク 恋の音▽*`
- `date`に誤りがあるケースがある
    - 例：`シティーハンター3`の1話から11話まで`1990`年に放送されたことになっているが、実際には`1989`年が正しいと思われる
- `acname`と`ccname`の対応関係を、文字列の類似度だけから機械的に判断することはできない
    - アニメ作品名と原作マンガ作品名が大きく異なることがある：
        - 例：アニメ`GS美神`と、その原作マンガ`ゴーストスイーパー美神 極楽大作戦!!`
        - 例：アニメ`ハヤテのごとく! Hayate the combat buttler CAN'T TAKE MY EYES OFF YOU`と、その原作マンガ`ハヤテのごとく!`
    - アニメ作品名とマンガ作品名が完全に一致していても、原作とは限らない
        - 例：アニメ`アソボット戦記 五九`の原作マンガは`アソボット戦記五九`ではなく`アソボット五九`（前者はアニメ放送開始後の特別編）
    - マンガ作品名に誤植の可能性がある：
        - 例：アニメ`ガンバリスト!駿`の原作マンガの候補として`ガンバ!Fly high`（305話以降を収録）と`ガンバ!!Fly high`（`!`が一つ多い誤植？第1話から304話まで収録）があるが、原作として紐づけるべきは **誤植の疑いのある後者** 
- 同一アニメシリーズの中でも、特別回を別の`acid`として切り出すことがある
    - 例：アニメ`こちら葛飾区亀有公園前派出所`と`こちら葛飾区 亀有公園前 派出所 [スペシャル] 両津の 浅草リニューアル 大作戦!! ～あぁ 思い出の花やしき～`は`acid`が異なる

`mix_ce_ac.csv`は、マンガ各話とそのアニメ化作品についてまとめたファイルです。今後分析で用いる際は、以下に特に注意するべきでしょう。

- `cename`に約15%程度の欠損がある
- `acid`、`acname`、`asid`、`n_ae`、`first_date_ac`、`last_date_ac`に約76%の欠損がある
    - これは **アニメ化実績のないマンガ作品もデータに含まれている** ため
- `mix_ae_crt.csv`と同様の理由で1990年以前のアニメ化データが欠損している
    - この結果、アニメ化実績のあるマンガ作品はそうでない作品より平均的に`two_colored`獲得率が **少なく** なり、誤った結論を導く可能性がある
        - `two_colored`が多用されていたのは1980年代だが、ちょうどその時期のアニメ作品のデータが欠損しているため
    - この結果、アニメ化実績のあるマンガ作品はそうでない作品より平均的に`first_date_cc`が新しくなり、これもまた誤った結論を導く可能性がある
